# Library

In [124]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

## [로드] 말소 데이터

In [35]:
ex_col_df = pd.read_excel('data/220819_22.06월기준_말소_컬럼.xlsx')
ex_col_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   컬럼명     42 non-null     object
dtypes: object(1)
memory usage: 464.0+ bytes


In [36]:
ex_col_df.head()

,컬럼명
0,차량관리번호
1,차량등록번호
2,본거지법정동코드
3,소유자구분
4,차종_등록정보


In [37]:
ex_col = ex_col_df['컬럼명']
len(ex_col)

42

In [38]:
# 20.8s
ex = pd.read_table('data/220819_22.06월기준_말소.txt', header=42, names=ex_col, usecols=['차대번호', '차량말소YN'], encoding='cp949')
ex.columns

Index(['차대번호', '차량말소YN'], dtype='object')

In [39]:
ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653054 entries, 0 to 3653053
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   차대번호    object
 1   차량말소YN  object
dtypes: object(2)
memory usage: 55.7+ MB


## [로드] 컬럼명 데이터

In [40]:
col_df = pd.read_table('data/컬럼명.txt', sep=',')
col= col_df.columns

In [41]:
col

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류'],
      dtype='object')

## [로드] 파일

In [42]:
# 22.3s
df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)
df.info()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_8084\3437007485.py:2: DtypeWarning: Columns (14,17,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table('data/220819_22.06월기준_4등급.txt', encoding='cp949', header=40, names=col)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1528586 entries, 0 to 1528585
Data columns (total 40 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   차량관리번호       1528586 non-null  object 
 1   차량등록번호       1528586 non-null  object 
 2   본거지법정동코드     1528586 non-null  int64  
 3   소유자구분        1528586 non-null  object 
 4   차종_등록정보      1528584 non-null  object 
 5   용도           1528586 non-null  object 
 6   차대번호         1528586 non-null  object 
 7   최초등록일자       1528586 non-null  int64  
 8   차량연식         1528586 non-null  int64  
 9   차량제작일자       1358925 non-null  float64
 10  검사유효일        1528574 non-null  float64
 11  제원관리번호       1528586 non-null  object 
 12  배출가스인증번호     1405585 non-null  object 
 13  배출가스등급       1528586 non-null  int64  
 14  저공해조치종류      195 non-null      object 
 15  저공해조치구조변경YN  1528586 non-null  object 
 16  차량말소YN       1528586 non-null  object 
 17  취득일자         1435514 non-null  object 
 18  취득

# 전처리

## 차량말소 정보 추가

In [43]:
df['차량말소YN'].unique()

array(['N'], dtype=object)

In [44]:
df2 = df.merge(ex, on='차대번호', how='left')
df2['차량말소YN_y'].unique()

array([nan, 'Y'], dtype=object)

In [45]:
df3 = df2[df2['차량말소YN_y'].isnull() == True]
df3.shape

(1528579, 41)

## 연료 정보 수정
- 엔진형식, 차명으로 확인

In [46]:
f = df3.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1167606
5,휘발유,334514
1,LPG(액화석유가스),26419
3,기타연료,38
0,CNG(압축천연가스),1
4,알코올,1


In [47]:
g = df3.groupby('연료', as_index=False)['차대번호'].groups
g['기타연료']

Int64Index([  38295,   73224,  103131,  105842,  113699,  121116,  150211,
             162707,  373733,  431838,  433717,  452642,  527072,  528529,
             539486,  560436,  563710,  684873,  723391,  743410,  760045,
             760597,  870040,  922854,  936173,  953452, 1064457, 1094648,
            1095179, 1124780, 1178312, 1221151, 1303029, 1353597, 1359097,
            1368257, 1408956, 1525597],
           dtype='int64')

In [48]:
# B5 : 휘발유
# G16SF : 휘발유
# H20MR : 휘발유
# MSV3 : 휘발유
# 01821300581 : 
# G15CF : 휘발유
# G4AJ : 휘발유
df3.loc[df3['연료'] == '기타연료', '엔진형식'].unique()

array(['B5', 'G16SF', nan, 'H20MR', 'MSV3', '01821300581', 'G15CF',
       'G4AJ', '-'], dtype=object)

In [49]:
df3.loc[df3['엔진형식'] == 'B5', '연료'] = '휘발유'
df3.loc[df3['엔진형식'] == 'G16SF', '연료'] = '휘발유'
df3.loc[df3['엔진형식'] == 'H20MR', '연료'] = '휘발유'
df3.loc[df3['엔진형식'] == 'MSV3', '연료'] = '휘발유'
df3.loc[df3['엔진형식'] == 'G15CF', '연료'] = '휘발유'
df3.loc[df3['엔진형식'] == 'G4AJ', '연료'] = '휘발유'
df3[df3['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
103131,도요다크라운로얄싸롱,NaN
150211,스텔라오토매틱,NaN
373733,벤즈 230E,NaN
433717,벤즈230E,01821300581
760045,에어스트림탐지견수송트레일러,-


In [50]:
df3.loc[df3['차명'] == '도요다크라운로얄싸롱', '연료'] = '휘발유'
df3.loc[df3['차명'] == '스텔라오토매틱', '연료'] = '휘발유'
df3.loc[df3['차명'] == '벤즈 230E', '연료'] = '휘발유'
df3.loc[df3['차명'] == '벤즈230E', '연료'] = '휘발유'
df3[df3['연료'] == '기타연료'][['차명', '엔진형식']]

,차명,엔진형식
760045,에어스트림탐지견수송트레일러,-


In [51]:
f = df3.groupby('연료', as_index=False)['차대번호'].count().sort_values('차대번호', ascending=False)
f

,연료,차대번호
2,경유,1167606
5,휘발유,334551
1,LPG(액화석유가스),26419
0,CNG(압축천연가스),1
3,기타연료,1
4,알코올,1


In [52]:
f.columns = ['연료', '4등급']
f

,연료,4등급
2,경유,1167606
5,휘발유,334551
1,LPG(액화석유가스),26419
0,CNG(압축천연가스),1
3,기타연료,1
4,알코올,1


## [excel] 연료별 통계

In [53]:
f.to_excel('analysis/[G4]연료별_차대번호_통계.xlsx', index=False, encoding='utf-8')

## 지역별 수량 통계

## [로드] 법정동코드

In [54]:
code = pd.read_excel('data/지역구분코드(수정).xlsx')
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ad1     229 non-null    object
 1   code1   229 non-null    int64 
 2   ad2     229 non-null    object
 3   code2   229 non-null    int64 
 4   code3   229 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 9.1+ KB


In [55]:
code = code[['ad1', 'code1']].drop_duplicates(['ad1'], keep='first').reset_index(drop=True)
code

,ad1,code1
0,서울특별시,11
1,부산광역시,26
2,대구광역시,27
3,인천광역시,28
4,광주광역시,29
5,대전광역시,30
6,울산광역시,31
7,세종특별자치시,36
8,경기도,41
9,강원도,42


### 법정동 코드 추가

In [56]:
df['본거지법정동코드'].head()

0    1144010700
1    2650010100
2    4729025024
3    4117110100
4    4711331026
Name: 본거지법정동코드, dtype: int64

In [57]:
df['본거지법정동코드'].isnull().sum()

0

In [58]:
df['지역코드'] = df['본거지법정동코드'].astype('str').str[:2]
df['지역코드'] = df['지역코드'].astype('int')

In [59]:
df['지역코드'].head()

0    11
1    26
2    47
3    41
4    47
Name: 지역코드, dtype: int32

In [60]:
code['code1'].head()

0    11
1    26
2    27
3    28
4    29
Name: code1, dtype: int64

In [61]:
df = df.merge(code, left_on='지역코드', right_on='code1', how='left')
df[['지역코드', 'ad1']].head()

,지역코드,ad1
0,11,서울특별시
1,26,부산광역시
2,47,경상북도
3,41,경기도
4,47,경상북도


In [62]:
df['ad1'].isnull().sum()

0

In [63]:
l = df.groupby(['ad1'], as_index=False)['차대번호'].count()
l

,ad1,차대번호
0,강원도,57012
1,경기도,367755
2,경상남도,117829
3,경상북도,117678
4,광주광역시,39849
5,대구광역시,69791
6,대전광역시,42190
7,부산광역시,77835
8,서울특별시,180530
9,세종특별자치시,8193


In [64]:
l.columns = ['지역', '2등급']
l

,지역,2등급
0,강원도,57012
1,경기도,367755
2,경상남도,117829
3,경상북도,117678
4,광주광역시,39849
5,대구광역시,69791
6,대전광역시,42190
7,부산광역시,77835
8,서울특별시,180530
9,세종특별자치시,8193


### [excel] 지역별 수량 통계

In [65]:
l.to_excel('analysis/[G4]지역별_차대번호_통계.xlsx', index=False, encoding='utf-8')

## [출력]

In [32]:
# 30.2s
df.to_csv('data/220819_22.06월기준_4등급(수정).csv', encoding='cp949')

## 지역별 연료별 통계

In [305]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '지역코드', 'ad1', 'code1', '총중량_분류', '배기량_분류'],
      dtype='object')

In [310]:
statics_by_local_fuel = df.groupby(['ad1', '연료'])['차대번호'].count()
statics_by_local_fuel

ad1      연료         
강원도      LPG(액화석유가스)       806
         경유              47557
         휘발유              8649
경기도      LPG(액화석유가스)      6971
         경유             271445
         기타연료                2
         휘발유             89337
경상남도     LPG(액화석유가스)      1044
         경유              97822
         기타연료                1
         휘발유             18962
경상북도     LPG(액화석유가스)      1471
         경유              96365
         기타연료                7
         휘발유             19835
광주광역시    LPG(액화석유가스)       721
         경유              30708
         휘발유              8420
대구광역시    LPG(액화석유가스)      1675
         경유              51173
         기타연료               24
         휘발유             16919
대전광역시    LPG(액화석유가스)      1248
         경유              30365
         휘발유             10577
부산광역시    LPG(액화석유가스)      1517
         경유              58875
         기타연료                1
         휘발유             17442
서울특별시    LPG(액화석유가스)      4439
         경유             112475
         기타연료     

In [312]:
statics_by_local_fuel2 = statics_by_local_fuel.unstack(['연료'])
statics_by_local_fuel2

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
ad1,,,,,,
강원도,NaN,806.0,47557.0,NaN,NaN,8649.0
경기도,NaN,6971.0,271445.0,2.0,NaN,89337.0
경상남도,NaN,1044.0,97822.0,1.0,NaN,18962.0
경상북도,NaN,1471.0,96365.0,7.0,NaN,19835.0
광주광역시,NaN,721.0,30708.0,NaN,NaN,8420.0
대구광역시,NaN,1675.0,51173.0,24.0,NaN,16919.0
대전광역시,NaN,1248.0,30365.0,NaN,NaN,10577.0
부산광역시,NaN,1517.0,58875.0,1.0,NaN,17442.0
서울특별시,NaN,4439.0,112475.0,2.0,NaN,63614.0


In [313]:
statics_by_local_fuel2.to_excel('analysis/[G4][법정동코드]지역_연료별_통계.xlsx', encoding='utf-8')

# 분석

## 배출가스인증번호 내역 확인

In [164]:
df['배출가스인증번호'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1528586 entries, 0 to 1528585
Series name: 배출가스인증번호
Non-Null Count    Dtype 
--------------    ----- 
1405585 non-null  object
dtypes: object(1)
memory usage: 55.6+ MB


In [165]:
df['배출가스인증번호'].isnull().sum()

123001

In [166]:
df['배출가스인증번호'].value_counts()

7MY-HD-14-68      268972
8MY-KM-13-19       93969
6MY-KM-14-61       76371
9MY-KM-14-41       67493
7MY-HD-14-46       60364
                   ...  
8MY-WI-0M-24           1
AMY-PPE-14-225         1
8MY-SN-0M-24           1
7MY-SK-24-03           1
AMY-PPE-14-800         1
Name: 배출가스인증번호, Length: 304, dtype: int64

In [167]:
temp = df['배출가스인증번호'].value_counts().reset_index()
temp

,index,배출가스인증번호
0,7MY-HD-14-68,268972
1,8MY-KM-13-19,93969
2,6MY-KM-14-61,76371
3,9MY-KM-14-41,67493
4,7MY-HD-14-46,60364
...,...,...
299,8MY-WI-0M-24,1
300,AMY-PPE-14-225,1
301,8MY-SN-0M-24,1
302,7MY-SK-24-03,1


In [90]:
# 제원정보 배출가스인증번호 현황
temp.to_csv('analysis/[G4][배출가스인증번호]유니크값_통계.csv', encoding='cp949')

### 12\. [e]배출가스인증번호

In [91]:
# - 정보
#     - 총 : 1528586
#     - null : 123001
#     - 정상
#         - 2005.01.04 이후 표기 방식([숫자문자]{3}-[문자]{2}-[숫자]{2}-[숫자]{2}, 12자리) : 1152680
#         - 2005.01.04 이전 표기 방식([숫자문자]{3}-[문자]{2}-[숫자]{2}, 9자리) : 230867
#     - 오류(null 값 제외) : 22038
#       - 예시
#           - 163      TY87-배K-02
#           - 173    7MY-KJ-0M-24
#           - 216      TY87-배K-03
#           - 236      TY87-배K-10
#           - 293      TY87-배K-03

In [92]:
# 2005년 이후 표기 방식 중 길이가 12자리가 아닌 값 : 3018
#     - [max] 차량연식 : 2011
# 2005년 이전 표기 방식 중 길이가 9자리가 아닌 값 : 2

##### 빈칸 처리

In [315]:
gas_df = df.copy()
gas_df.head()

,차량관리번호,차량등록번호,본거지법정동코드,소유자구분,차종_등록정보,용도,차대번호,최초등록일자,차량연식,차량제작일자,...,원산지명,삭제YN,제원승인일자,구동형식,변속기종류,지역코드,ad1,code1,총중량_분류,배기량_분류
0,KMFZCY7JBCU784032-01,97구4490,1144010700,법인,화물,자가용,KMFZCY7JBCU784032,20110817,2012,20110811.0,...,대한민국,N,20110524,NaN,자동,11,서울특별시,11,3.0ton 이상 3.5ton 미만,2.0L 이상 3.0L 미만
1,KMHJN81VP7U578447-01,11너2154,2650010100,주민,승용,자가용,KMHJN81VP7U578447,20070102,2007,20061104.0,...,대한민국,N,20060621,NaN,자동,26,부산광역시,26,1.5ton 이상 2.0ton 미만,1.6L 이상 2.0L 미만
2,KPACE4AB1AP070600-01,96어5714,4729025024,주민,화물,자가용,KPACE4AB1AP070600,20091221,2010,20091210.0,...,대한민국,N,20090513,NaN,자동,47,경상북도,47,2.5ton 이상 3.0ton 미만,1.6L 이상 2.0L 미만
3,KMHNU81WP7U009391-01,11가1154,4117110100,사업자,승용,자가용,KMHNU81WP7U009391,20070212,2007,20070207.0,...,대한민국,N,20060821,NaN,자동,41,경기도,41,2.5ton 이상 3.0ton 미만,2.0L 이상 3.0L 미만
4,KMHDT41TP9U638425-01,22보7099,4711331026,주민,승용,자가용,KMHDT41TP9U638425,20081128,2009,20081123.0,...,대한민국,N,20080729,NaN,수동,47,경상북도,47,1.5ton 이상 2.0ton 미만,1.0L 이상 1.6L 미만


In [316]:
gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호']

103131      
373733      
528751      
662671      
755942      
917544      
921939      
1082924     
1085965     
1346008     
1419281     
1528483     
Name: 배출가스인증번호, dtype: object

In [317]:
gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호'].shape

(12,)

In [318]:
blank_index = gas_df[gas_df['배출가스인증번호'] == ' ']['배출가스인증번호'].index
blank_index

Int64Index([ 103131,  373733,  528751,  662671,  755942,  917544,  921939,
            1082924, 1085965, 1346008, 1419281, 1528483],
           dtype='int64')

In [319]:
# 빈칸을 포함한 예시
gas_df.iloc[103131, 12]

' '

In [320]:
gas_df.loc[blank_index, '배출가스인증번호'] = np.nan
gas_df.iloc[103131, 12]

nan

In [321]:
gas_df['배출가스인증번호'].isnull().sum()

123013

In [322]:
df.iloc[:, 12].value_counts()

7MY-HD-14-68      268972
8MY-KM-13-19       93969
6MY-KM-14-61       76371
9MY-KM-14-41       67493
7MY-HD-14-46       60364
                   ...  
8MY-WI-0M-24           1
AMY-PPE-14-225         1
8MY-SN-0M-24           1
7MY-SK-24-03           1
AMY-PPE-14-800         1
Name: 배출가스인증번호, Length: 304, dtype: int64

In [323]:
df.iloc[:, 12].isnull().sum()

123001

In [324]:
df[df['배출가스인증번호'].isnull() == True]['배출가스인증번호'].head()

29    NaN
36    NaN
42    NaN
63    NaN
67    NaN
Name: 배출가스인증번호, dtype: object

In [325]:
# 2005.01.04 이후 표기 방식 : 1152680
count = 0
p121 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}[-][0-9]{2}')
idx121 = []
for i, one in tqdm(enumerate(gas_df.iloc[:, 12].to_numpy())):
    if p121.search(str(one)):
        idx121.append(i)
        count += 1
count

1528586it [00:01, 891696.05it/s]


1152680

In [326]:
# 2005.01.04 이후 표기 방식이 아닌 값 : 375906
count = 0
idx122 = []
for i, one in tqdm(enumerate(gas_df.iloc[:, 12].to_numpy())):
    if not p121.search(str(one)):
        idx122.append(i)
        count += 1
count

1528586it [00:01, 1037672.90it/s]


375906

In [327]:
gas_df.iloc[:, 12].reindex(idx122)[:5]

5     VMY-KM-38
21    RMY-HD-20
28    KMY-HD-03
29          NaN
36          NaN
Name: 배출가스인증번호, dtype: object

In [328]:
# 2005.01.04 이전 배출가스 표기 방식 : 230867
count = 0
p123 = re.compile('[a-zA-Z0-9]{3}[-][a-zA-Z]{2}[-][0-9]{2}')
idx123 = []
for i, one in tqdm(enumerate(gas_df.iloc[idx122, 12].to_numpy())):
    if p123.search(str(one)):
        idx123.append(idx122[i])
        count += 1
count

375906it [00:00, 850457.06it/s]


230867

In [329]:
gas_df.iloc[idx123, [7, 8, 9, 17, 12]][:5]

,최초등록일자,차량연식,차량제작일자,취득일자,배출가스인증번호
5,20000609,2000,20000529.0,20030219.0,VMY-KM-38
21,19990608,1999,19990513.0,20040401.0,RMY-HD-20
28,19910820,1991,NaN,NaN,KMY-HD-03
46,19991213,2000,19991202.0,20150914.0,RMY-HD-20
62,19970522,1997,19970310.0,NaN,MMY-HD-11


In [330]:
# null 값 : 123013
idx124 = gas_df[gas_df['배출가스인증번호'].isnull() == True]['배출가스인증번호'].index
len(idx124)

123013

In [331]:
gas_df.iloc[:, 12].reindex(idx124)[:5]

29    NaN
36    NaN
42    NaN
63    NaN
67    NaN
Name: 배출가스인증번호, dtype: object

In [332]:
# 2005.01.04 이후 표기 방식이 아닌 값 중 null 값 제외한 오류 값 : 22026
idx125 = sorted(set(idx122) - set(idx124) - set(idx123))
len(idx125)

22026

In [333]:
gas_df.iloc[:, 12].reindex(idx125)[:5]

163      TY87-배K-02
173    7MY-KJ-0M-24
216      TY87-배K-03
236      TY87-배K-10
293      TY87-배K-03
Name: 배출가스인증번호, dtype: object

In [334]:
# 2005년 이상 배출가스인증번호 데이터
gas_temp = gas_df.iloc[idx123, [7, 8, 9, 17, 12]]
gas_temp[gas_temp['차량연식'] >= 2005].head()

,최초등록일자,차량연식,차량제작일자,취득일자,배출가스인증번호
582,20100209,2010,20100205.0,20120910.0,AMY-JL-13-9
810,20070816,2007,20070816.0,20100423.0,6MY-PA-14-4
1629,20100409,2010,20100205.0,20180423.0,AMY-JL-13-9
2087,20100511,2010,20100506.0,20190611.0,AMY-JL-13-9
2142,20100504,2010,20091207.0,20180430.0,8MY-PA-14-3


In [335]:
# 2005년 이상년도에서 맞지않는 배출가스인증번호 샘플 : 3018
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12)]

,최초등록일자,차량연식,차량제작일자,취득일자,배출가스인증번호
582,20100209,2010,20100205.0,20120910.0,AMY-JL-13-9
810,20070816,2007,20070816.0,20100423.0,6MY-PA-14-4
1629,20100409,2010,20100205.0,20180423.0,AMY-JL-13-9
2087,20100511,2010,20100506.0,20190611.0,AMY-JL-13-9
2142,20100504,2010,20091207.0,20180430.0,8MY-PA-14-3
...,...,...,...,...,...
1524691,20100409,2010,20100212.0,20191108.0,9MY-JL-14-4
1524870,20100108,2010,20091102.0,20110119.0,AMY-JL-13-9
1525658,20090415,2009,20081208.0,20190930.0,8MY-PA-14-3
1527579,20100108,2010,20091223.0,20100405.0,6MY-PA-14-4


In [336]:
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12 )].shape

(3018, 5)

In [337]:
gas_temp.loc[(gas_temp['차량연식'] >= 2005) & (gas_temp['배출가스인증번호'].str.len() != 12 ), '차량연식'].max()

2011

In [338]:
# 2005년 미만 년도에서 맞지않는 배출가스인증번호 샘플 : 2
gas_temp.loc[(gas_temp['차량연식'] < 2005) & (gas_temp['배출가스인증번호'].str.len() != 9 )]

,최초등록일자,차량연식,차량제작일자,취득일자,배출가스인증번호
747837,20091013,1999,19991231.0,10091009.0,9MY-PP-14-W38
1424917,20090917,1999,20081124.0,20201106.0,9MY-PP-14-W38


In [382]:
num_by_gas_num = gas_df.groupby('배출가스인증번호')['차대번호'].count().reset_index()
num_by_gas_num

,배출가스인증번호,차대번호
0,7MY-DC-24-72,1
1,7MY-HD-24-30,8
2,8MY-DC-24-12,1
3,3MY-FK-08,1067
4,4MY-AD-20,38
...,...,...
298,XMY-HD-47,924
299,XMY-HD-48,364
300,XMY-HD-55,2532
301,XMY-KM-47,3217


In [384]:
num_by_gas_num.to_excel('analysis/[G4][배출가스인증번호]유니크값_통계.xlsx', encoding='utf-8')

In [385]:
gas_df['배출가스인증번호'].isnull().sum()

123013

#### 오류 배출가스인증번호 유형 통계

In [339]:
# - 총 오류 샘플 수 : 22026
#     - 1번째 부분 3자리가 아닌 경우 : 17842
#         - 빈 값(' ')인 경우 : 0
#         - 바('-') 기호가 없는 경우 : 6
#         - 1자리인 경우 : 0
#         - 2자리인 경우 : 0
#         - 4자리인 경우 : 17836
#     - 2번째 부분 2자리가 아닌 경우 : 2827
#         - 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
#         - 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2827
#     - 2번째 부분 2자리이면서 숫자만 있는 경우 : 0
#     - 3번째 부분 2자리가 아닌 경우 : 1
#         - 3번째 부분 1자리인 경우(XXX-XX-n-XX) : 1
#     - 3번째 부분 2자리이면서 문자가 있는 경우 : 1356
#     - 4번째 부분 2자리가 아닌 경우 : 0(1999)
#         - 중복 오류 : 1999
#     - 4번째 부분 2자리이면서 문자가 있는 경우 : 0(19193)
#         - 중복 오류 : 19193

In [340]:
gas_df.iloc[idx125[:5], 12]

163      TY87-배K-02
173    7MY-KJ-0M-24
216      TY87-배K-03
236      TY87-배K-10
293      TY87-배K-03
Name: 배출가스인증번호, dtype: object

In [341]:
gas_temp2 = gas_df.iloc[idx125, 12]
gas_temp2.head()

163      TY87-배K-02
173    7MY-KJ-0M-24
216      TY87-배K-03
236      TY87-배K-10
293      TY87-배K-03
Name: 배출가스인증번호, dtype: object

##### 1번째 자리 오류 유형

In [342]:
# 1번째 부분 3자리가 아닌 경우 : 17842
#     - 빈 값(' ')인 경우 : 0
#     - 바('-') 기호가 없는 경우 : 6
#     - 1자리인 경우 : 0
#     - 2자리인 경우 : 0
#     - 4자리인 경우 : 17836

In [343]:
idx126 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) != 3:
            idx126.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 447168.84it/s]


17842

In [344]:
gas_df.iloc[idx126, 12]

163        TY87-배K-02
216        TY87-배K-03
236        TY87-배K-10
293        TY87-배K-03
331        TY87-배K-02
              ...    
1528154    TY87-배K-02
1528199    TY87-배K-10
1528407    TY87-배K-10
1528409    TY87-배K-03
1528552    TY87-배K-10
Name: 배출가스인증번호, Length: 17842, dtype: object

In [345]:
gas_df.iloc[idx126[0], 12]

'TY87-배K-02'

In [346]:
count = 0
idx126_1 = []
for i, one in enumerate(gas_df.iloc[idx126, 12]):
    if one == ' ':
        idx126_1.append(gas_df.iloc[idx126, 12].index[i])
        count += 1
count

0

In [347]:
gas_df.iloc[idx126_1, 12]

Series([], Name: 배출가스인증번호, dtype: object)

In [348]:
count = 0
idx126_2 = []
for i, one in enumerate(gas_df.iloc[idx126, 12]):
    if '-' not in one:
        idx126_2.append(gas_df.iloc[idx126, 12].index[i])
        count += 1
count

6

In [349]:
gas_df.iloc[list(set(idx126_2) - set(idx126_1)), 12]

67005      C135
79069      C135
1236747    C135
264364     C135
1189933    C135
1377263    C135
Name: 배출가스인증번호, dtype: object

In [350]:
len(list(set(idx126_2) - set(idx126_1)))

6

In [351]:
idx126_3 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 1:
            idx126_3.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 959621.69it/s]


0

In [352]:
gas_df.iloc[idx126_3[:5], 12]

Series([], Name: 배출가스인증번호, dtype: object)

In [353]:
set(idx126_1) == set(idx126_3)

True

In [354]:
idx126_4 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 2:
            idx126_4.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 815606.43it/s]


0

In [355]:
idx126_5 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[0]) == 4:
            idx126_5.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 466398.46it/s]


17842

In [356]:
len(set(idx126_5) - set(idx126_1) - set(idx126_2))

17836

##### 2번째 자리 오류 유형

In [357]:
# 2번째 부분 2자리가 아닌 경우 : 2827
#   - 2번째 부분 1자리인 경우(XXX-n-XX-XX) : 0
#   - 2번째 부분 3자리인 경우(XXX-nnn-XX-XX) : 2827
# 2번째 부분 2자리이면서 숫자만 있는 경우 : 0

In [358]:
idx127 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) != 2:
            idx127.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 763861.82it/s]


2827

In [359]:
len(set(idx127) - set(idx126))

2827

In [360]:
gas_df.iloc[idx127[:5], 12]

547      FMY-VTK-14-1
639      FMY-VTK-14-1
1812    7MY-VTK-24-01
2178    7MY-VTK-24-01
2253     FMY-VTK-14-3
Name: 배출가스인증번호, dtype: object

In [361]:
idx127_1 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 1:
            idx127_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 846935.64it/s]


0

In [362]:
gas_df.iloc[idx127_1[:5], 12]

Series([], Name: 배출가스인증번호, dtype: object)

In [363]:
idx127_3 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 3:
            idx127_3.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 687967.68it/s]


2827

In [364]:
gas_df.iloc[idx127_3[:5], 12]

547      FMY-VTK-14-1
639      FMY-VTK-14-1
1812    7MY-VTK-24-01
2178    7MY-VTK-24-01
2253     FMY-VTK-14-3
Name: 배출가스인증번호, dtype: object

In [365]:
idx127_4 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[1]) == 4:
            idx127_4.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 957433.75it/s]


0

In [366]:
# 2번째 부분 2자리이면서 숫자만 있는 경우 : 0
idx127_5 = []
p127_5 = re.compile('[0-9]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p127_5.search(str(one).split('-')[1]):
            idx127_5.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 535731.83it/s]


0

##### 3번째 자리 오류 유형

In [367]:
# 3번째 부분 2자리가 아닌 경우 : 1
#   - 3번째 부분 1자리인 경우(XXX-XX-n-XX) : 1
# 3번째 부분 2자리이면서 문자가 있는 경우 : 1357

In [368]:
idx128 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[2]) != 2:
            idx128.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 956878.41it/s]


1

In [369]:
gas_df.iloc[idx128, 12]

1470564    NMY-JK-S-107
Name: 배출가스인증번호, dtype: object

In [370]:
len(set(idx128) - set(idx126) - set(idx127))

1

In [371]:
# 3번째 부분 2자리이면서 문자가 있는 경우 : 1357
idx128_1 = []
p128_1 = re.compile('[a-zA-Z가-힣]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p128_1.search(str(one).split('-')[1]):
            idx128_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 328611.05it/s]


19193

In [372]:
len(set(idx128_1) - set(idx126) - set(idx127) - set(idx128))

1356

In [373]:
gas_df.iloc[list(set(idx128_1) - set(idx126) - set(idx127) - set(idx128))[:5], 12]

417796     6MY-SS-0M-24
286727     7MY-HT-0M-24
1499144    7MY-JI-0M-24
1130509    7MY-HS-0M-24
1363981    7MY-SJ-0M-24
Name: 배출가스인증번호, dtype: object

##### 4번째 자리 오류 유형

In [374]:
# 4번째 부분 2자리가 아닌 경우 : 0(1999)
#   - 중복 오류 : 1999
# 4번째 부분 2자리이면서 문자가 있는 경우 : 0(19193)

In [375]:
idx129 = []
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if len(str(one).split('-')[3]) != 2:
            idx129.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 688188.05it/s]


1999

In [376]:
gas_df.iloc[idx129[:5], 12]

547     FMY-VTK-14-1
639     FMY-VTK-14-1
2253    FMY-VTK-14-3
2338    FMY-VTK-14-1
4796    FMY-VTK-14-1
Name: 배출가스인증번호, dtype: object

In [377]:
# 중복 추출된 샘플 제거
len(set(idx129) - set(idx126) - set(idx127) - set(idx128))

0

In [378]:
# 4번째 부분 2자리이면서 문자가 있는 경우 : 0
idx129_1 = []
p129_1 = re.compile('[a-zA-Z가-힣]+')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if (len(str(one).split('-')[1]) == 2) and p129_1.search(str(one).split('-')[1]):
            idx129_1.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 331535.67it/s]


19193

In [379]:
len(set(idx129_1) - set(idx126) - set(idx127) - set(idx128_1))

0

##### 한글이 포함된 경우

In [380]:
idx1210 = []
p1210 = re.compile('[가-힣]')
count = 0
for i, one in tqdm(enumerate(gas_temp2.to_numpy())):
    try:
        if p1210.search(one):
            idx1210.append(gas_temp2.index[i])
            count += 1
    except:
        pass
count

22026it [00:00, 524186.86it/s]


17836

In [381]:
len(set(idx1210) - set(idx129) - set(idx126) - set(idx127) - set(idx128))

0

### 배출가스인증번호 미부여 현황
- 필요한 컬럼 : 배출가스인증번호, 차량연식, 차량제작일자, 차명, 차종_제원정보, 차종분류, 차종유형, 자동차형식, 제작사명, 연료

In [235]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '지역코드', 'ad1', 'code1'],
      dtype='object')

In [236]:
gas_df.loc[gas_df['배출가스인증번호'].isnull() == True, ['배출가스인증번호', '차량연식', '차량제작일자', '차명', '차종_제원정보', '차종분류', '차종유형', '제작사명', '연료']].head()

,배출가스인증번호,차량연식,차량제작일자,차명,차종_제원정보,차종분류,차종유형,제작사명,연료
29,NaN,1989,19891004.0,로얄살롱 2.0A.T,승용,일반,중형,대우자동차(주),휘발유
36,NaN,1997,19970130.0,티코(TICO),승용,일반,경형,대우중공업(주),휘발유
42,NaN,1996,NaN,프린스1.8DOHCAT,승용,일반,중형,대우자동차(주),휘발유
63,NaN,1993,NaN,타우너,화물,밴,경형,아시아자동차공업(주),휘발유
67,NaN,1994,NaN,프린스1.8AT,승용,일반,중형,대우자동차(주),휘발유


In [239]:
gas_df.loc[gas_df['배출가스인증번호'].isnull() == True, ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류']].shape

(123013, 40)

In [240]:
null_num_df = gas_df.loc[gas_df['배출가스인증번호'].isnull() == True, ['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류']]

In [242]:
null_num_df.to_csv('analysis/[G4][배출가스인증번호]미부여_샘플.csv', encoding='cp949')

## 배출가스인증번호별 통계

In [442]:
gas_df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '차대번호',
       '최초등록일자', '차량연식', '차량제작일자', '검사유효일', '제원관리번호', '배출가스인증번호', '배출가스등급',
       '저공해조치종류', '저공해조치구조변경YN', '차량말소YN', '취득일자', '취득금액', '차명', '차종_제원정보',
       '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', '총중량', '적재중량', '엔진출력',
       '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '제원승인일자', '구동형식',
       '변속기종류', '지역코드', 'ad1', 'code1', '총중량_분류', '배기량_분류'],
      dtype='object')

In [443]:
per_inspect = pd.read_csv('data/220819_정기검사.csv', encoding='cp949')

In [444]:
per_detail_inspect = pd.read_csv('data/220819_정밀검사.csv', encoding='cp949')

C:\Users\kbjung\AppData\Local\Temp\ipykernel_8084\2544207015.py:1: DtypeWarning: Columns (12,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  per_detail_inspect = pd.read_csv('data/220819_정밀검사.csv', encoding='cp949')


In [445]:
per_inspect.columns

Index(['검사소명', '정기검사일자', '검사종류', '검사접수번호', '통합접수번호', '차량번호', '차대번호', '차량형식번호',
       '차명', '차종', '차량용도', '차량연식', '차량연료', '주행거리', '차량길이', '차량너비', '차량높이',
       '엔진형식', '배기량', '엔진출력', '차량중량', '총중량', '정원', '비고', '최초등록일자', '제작일자',
       '검사전 유효기간 만료일', '관능검사판정', '관능부적합항목', '검사방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '이산화탄소값', '최대출력허용치', '최대출력측정값', '산소값', '공기온도', '공기압력',
       '검사판정', '재검사기간', '검사후 갱신유효기간', '관능검사원', '배출가스검사원', '책임검사원', '검사 판정시간',
       '자료구분코드'],
      dtype='object')

In [447]:
temp_pins = per_inspect[['차대번호', '검사방법', '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6']]

In [446]:
per_detail_inspect.columns

Index(['정밀검사소명', '정밀검사일자', '정밀검사종류', '정밀검사접수번호', '차량번호', '차대번호', '차량형식번호',
       '차명', '차종', '차량용도', '차량연식', '차량연료', '주행거리', '차량길이', '차량너비', '차량높이',
       '엔진형식', '원동기배기량', '엔진출력', '차량중량', '총중량', '정원', '비고', '최초등록일',
       '정밀검사유효기간', '관능검사판정', '관능검사부적합내용', '정밀검사측정방법', '무부하매연측정치1', '무부하매연허용치1',
       '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3',
       '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4',
       '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '이산화탄소값', '최대출력허용치', '최대출력측정값',
       '산소값', '공기온도', '공기압력', '정밀검사판정', '재검사기간', '정밀검사유효기간2', '관능검사등록자',
       '정밀검사등록자', '정밀검사소대표명', '기타내용1', '기타내용2', '기타내용3', '기타내용4', '기타내용5',
       '원본대장테이블', '정밀검사시간정보', '통합접수번호', '데이터생성일자', '무부하매연측정치6', '무부하매연허용치6',
       '무부하매연판정6', '수신파일명', '제작등록일자', '데이터구분', '삭제구분', '삭제사유1', '삭제사유2',
       '삭제사유3', '삭제사유4'],
      dtype='object')

In [449]:
temp_pdins = per_detail_inspect[['차대번호', '정밀검사측정방법', '무부하매연측정치1', '무부하매연허용치1', '무부하매연판정1', '무부하매연측정치2', '무부하매연허용치2', '무부하매연판정2', '무부하매연측정치3', '무부하매연허용치3', '무부하매연판정3', '무부하매연측정치4', '무부하매연허용치4', '무부하매연판정4', '무부하매연측정치5', '무부하매연허용치5', '무부하매연판정5', '무부하매연측정치6', '무부하매연허용치6', '무부하매연판정6']]

In [451]:
gas_df.shape

(1528586, 45)

In [452]:
tgas_df = gas_df.merge(temp_pins, on='차대번호', how='left')
tgas_df2 = tgas_df.merge(temp_pdins, on='차대번호', how='left')

(1562667, 83)

In [453]:
tgas_df.shape, tgas_df2.shape

((1540457, 64), (1562667, 83))

## 총중량으로 분류 현황

In [243]:
df['총중량'].unique()

array([ 3280.,  1965.,  2615., ..., 17190., 17340., 21690.])

In [245]:
type(df['총중량'][0])

numpy.float64

In [246]:
df['총중량'].isnull().sum()

1

In [249]:
df['총중량'] = df['총중량'].fillna(-1)

In [250]:
df['총중량'].isnull().sum()

0

In [251]:
weight_list = []
for i, one in tqdm(enumerate(df['총중량'])):
    if one == -1:
        weight_list.append('미기입')
    elif one < 1500:
        weight_list.append('1.5ton 미만')
    elif one >= 1500 and one < 2000:
        weight_list.append('1.5ton 이상 2.0ton 미만')
    elif one >= 2000 and one < 2500:
        weight_list.append('2.0ton 이상 2.5ton 미만')
    elif one >= 2500 and one < 3000:
        weight_list.append('2.5ton 이상 3.0ton 미만')
    elif one >= 3000 and one < 3500:
        weight_list.append('3.0ton 이상 3.5ton 미만')
    elif one >= 3500 and one < 4000:
        weight_list.append('3.5ton 이상 4.0ton 미만')
    elif one >= 4000 and one < 4500:
        weight_list.append('4.0ton 이상 4.5ton 미만')
    elif one >= 4500 and one < 5000:
        weight_list.append('4.5ton 이상 5.0ton 미만')
    elif one >= 5000 and one < 5500:
        weight_list.append('5.0ton 이상 5.5ton 미만')
    elif one >= 5500 and one < 6000:
        weight_list.append('4.0ton 이상 4.5ton 미만')
    elif one >= 6000 and one < 6500:
        weight_list.append('6.0ton 이상 6.5ton 미만')
    elif one >= 6500 and one < 7000:
        weight_list.append('6.5ton 이상 7.0ton 미만')
    elif one >= 7000 and one < 7500:
        weight_list.append('7.0ton 이상 7.5ton 미만')
    elif one >= 7500 and one < 8000:
        weight_list.append('7.5ton 이상 8.0ton 미만')
    elif one >= 8000 and one < 8500:
        weight_list.append('8.0ton 이상 8.5ton 미만')
    elif one >= 8500 and one < 9000:
        weight_list.append('8.5ton 이상 9.0ton 미만')
    elif one >= 9000 and one < 9500:
        weight_list.append('9.0ton 이상 9.5ton 미만')
    elif one >= 9500 and one < 10000:
        weight_list.append('9.5ton 이상 10.0ton 미만')
    elif one >= 10000:
        weight_list.append('10.0ton 이상')

1528586it [00:01, 1030027.78it/s]


In [252]:
len(weight_list)

1528586

In [253]:
df['총중량_분류'] = weight_list

In [255]:
total_weight_df = df.groupby('총중량_분류')['차대번호'].count().reset_index()
total_weight_df

,총중량_분류,차대번호
0,1.5ton 미만,198432
1,1.5ton 이상 2.0ton 미만,331013
2,10.0ton 이상,30469
3,2.0ton 이상 2.5ton 미만,240260
4,2.5ton 이상 3.0ton 미만,436565
5,3.0ton 이상 3.5ton 미만,262435
6,3.5ton 이상 4.0ton 미만,135
7,4.0ton 이상 4.5ton 미만,13997
8,4.5ton 이상 5.0ton 미만,23
9,5.0ton 이상 5.5ton 미만,891


In [256]:
total_weight_df.to_excel('analysis/[G4][총중량]분류_통계.xlsx', encoding='utf-8')

## 배기량으로 분류 현황

In [257]:
df['배기량'].value_counts()

2497.0     412725
1991.0     245078
2902.0     139727
2959.0      75053
1995.0      68705
            ...  
4701.0          1
3803.0          1
5958.0          1
15607.0         1
12412.0         1
Name: 배기량, Length: 627, dtype: int64

In [258]:
df['배기량'].isnull().sum()

4

In [259]:
df['배기량'] = df['배기량'].fillna(-1)

In [260]:
gas_volume_list = []
for i, one in tqdm(enumerate(df['배기량'])):
    if one == -1:
        gas_volume_list.append('미기입')
    elif one < 800:
        gas_volume_list.append('0.8L 미만')
    elif one >= 800 and one < 1000:
        gas_volume_list.append('0.8L 이상 1.0L 미만')
    elif one >= 1000 and one < 1600:
        gas_volume_list.append('1.0L 이상 1.6L 미만')
    elif one >= 1600 and one < 2000:
        gas_volume_list.append('1.6L 이상 2.0L 미만')
    elif one >= 2000 and one < 3000:
        gas_volume_list.append('2.0L 이상 3.0L 미만')
    elif one >= 3000 and one < 4000:
        gas_volume_list.append('3.0L 이상 4.0L 미만')
    elif one >= 4000 and one < 5000:
        gas_volume_list.append('4.0L 이상 5.0L 미만')
    elif one >= 5000 and one < 6000:
        gas_volume_list.append('5.0L 이상 6.0L 미만')
    elif one >= 6000 and one < 8000:
        gas_volume_list.append('6.0L 이상 8.0L 미만')
    elif one >= 8000 and one < 10000:
        gas_volume_list.append('8.0L 이상 10.0L 미만')
    elif one >= 10000:
        gas_volume_list.append('10.0L 이상')

1528586it [00:01, 960266.45it/s]


In [261]:
len(gas_volume_list)

1528586

In [262]:
df['배기량_분류'] = gas_volume_list

In [264]:
total_gas_volume_df = df.groupby(['배기량_분류'])['차대번호'].count().reset_index()
total_gas_volume_df

,배기량_분류,차대번호
0,0.8L 미만,28674
1,0.8L 이상 1.0L 미만,171
2,1.0L 이상 1.6L 미만,165914
3,1.6L 이상 2.0L 미만,510894
4,10.0L 이상,16847
5,2.0L 이상 3.0L 미만,755439
6,3.0L 이상 4.0L 미만,31434
7,4.0L 이상 5.0L 미만,1342
8,5.0L 이상 6.0L 미만,15493
9,6.0L 이상 8.0L 미만,711


In [265]:
total_gas_volume_df.to_excel('analysis/[G4][배기량]분류_통계.xlsx', encoding='utf-8')

## 차대번호로 분류

### wmi 코드로 분류

In [269]:
wmi = pd.read_excel('data/20190730_WMI CODE.xlsx', sheet_name='WMI CODE')
wmi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   WMI            690 non-null    object 
 1   MF CODE        690 non-null    object 
 2   Identifier     690 non-null    object 
 3   Manufacturer   690 non-null    object 
 4   Country        688 non-null    object 
 5   Vehicle Types  249 non-null    object 
 6   Description    54 non-null     object 
 7   참고             0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 43.2+ KB


In [294]:
df2 = df.copy()

In [295]:
df2['차대번호'].head()

0    KMFZCY7JBCU784032
1    KMHJN81VP7U578447
2    KPACE4AB1AP070600
3    KMHNU81WP7U009391
4    KMHDT41TP9U638425
Name: 차대번호, dtype: object

In [296]:
df2['wmi'] = df2['차대번호'].str[:3]

In [297]:
df2['wmi'].head()

0    KMF
1    KMH
2    KPA
3    KMH
4    KMH
Name: wmi, dtype: object

In [298]:
df2 = df2.merge(wmi, left_on='wmi', right_on='WMI', how='left')
df2[['차대번호', 'wmi', 'WMI']].head()

,차대번호,wmi,WMI
0,KMFZCY7JBCU784032,KMF,KMF
1,KMHJN81VP7U578447,KMH,KMH
2,KPACE4AB1AP070600,KPA,KPA
3,KMHNU81WP7U009391,KMH,KMH
4,KMHDT41TP9U638425,KMH,KMH


In [299]:
df2['WMI'].isnull().sum()

75234

In [300]:
df2['Manufacturer'].isnull().sum()

75234

In [301]:
df2['Manufacturer'] = df2['Manufacturer'].fillna(-1)


In [302]:
manu_df = df2.groupby('Manufacturer')['차대번호'].count().reset_index()
manu_df

,Manufacturer,차대번호
0,-1,75234
1,Alfa Romeo,1
2,Audi,2106
3,Audi Hungary,1
4,Austin-Rover,83
...,...,...
101,Volvo Cars,3873
102,Volvo Trucks,2793
103,XLE Scania Netherlands,675
104,smart,3


In [303]:
manu_df = manu_df.sort_values('차대번호', ascending=False).reset_index(drop=True)
manu_df

,Manufacturer,차대번호
0,Hyundai,887094
1,Kia,358332
2,Daewoo General Motors South Korea,116241
3,-1,75234
4,SsangYong,31201
...,...,...
101,Honda,1
102,Mazda USA,1
103,Matra,1
104,Alfa Romeo,1


In [304]:
manu_df.to_excel('analysis/[G4][차대번호]제작사별_통계.xlsx', encoding='utf-8')

### 6\. [e]차대번호

In [ ]:
# - 오류
#     - 1\. null 값 : 0
#     - 2\. 정상(17자리, 영문 포함) : 1525955
#     - 3\. 17자리, 영문X : 1
#     - 4\. 17자리X, 영문 : 57
#     - 5\. 17자리X, 영문X : 2573

In [386]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1528586 entries, 0 to 1528585
Data columns (total 54 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   차량관리번호         1528586 non-null  object 
 1   차량등록번호         1528586 non-null  object 
 2   본거지법정동코드       1528586 non-null  int64  
 3   소유자구분          1528586 non-null  object 
 4   차종_등록정보        1528584 non-null  object 
 5   용도             1528586 non-null  object 
 6   차대번호           1528586 non-null  object 
 7   최초등록일자         1528586 non-null  int64  
 8   차량연식           1528586 non-null  int64  
 9   차량제작일자         1358925 non-null  float64
 10  검사유효일          1528574 non-null  float64
 11  제원관리번호         1528586 non-null  object 
 12  배출가스인증번호       1405585 non-null  object 
 13  배출가스등급         1528586 non-null  int64  
 14  저공해조치종류        195 non-null      object 
 15  저공해조치구조변경YN    1528586 non-null  object 
 16  차량말소YN         1528586 non-null  object 
 17  취득일자    

In [387]:
df2['차대번호'].isnull().sum()

0

In [ ]:
len('KMFZCY7JBCU784032')

17

#### 차대번호 정상 확인

In [389]:
# 17자리, 영문 -> 정상 : 1525955
count = 0
p61 = re.compile('[a-zA-Z]+')
idx61 = []
for i, one in tqdm(enumerate(df2.iloc[:, 6].to_numpy())):
    if len(str(one)) == 17 and p61.search(str(one)):
        idx61.append(i)
        count += 1
count

1528586it [00:02, 715477.40it/s]


1525955

In [390]:
# 정상에서 "I, O, Q" 포함한 값 : 1018
temp61 = df2.iloc[idx61, 6].to_numpy()
idx61_1 = []
for i, one in tqdm(enumerate(temp61)):
    if ('I' in one) or ('O' in one) or ('Q' in one):
        idx61_1.append(idx61[i])
len(idx61_1)

1525955it [00:00, 2098635.25it/s]


1018

In [391]:
df2.iloc[idx61_1, 6][:5]

711     WFOFXXGBBVTS14067
4089    IMECM53U7LA639275
4306    KNFSEOD428K331451
7352    IMECM53UXLA639285
8869    IMECM53U4LA619808
Name: 차대번호, dtype: object

In [392]:
# 17자리, 영문X  => 숫자만 : 1
count = 0
p62 = re.compile('[a-zA-Z]+')
idx62 = []
for i, one in tqdm(enumerate(df2.iloc[:, 6].to_numpy())):
    n = 0
    for j in str(one):
        if p62.search(j):
            n += 1
    if len(str(one)) == 17 and n == 0:
        idx62.append(i)
        count += 1
count

1528586it [00:12, 127145.87it/s]


1

In [393]:
df2.iloc[:, 6].reindex(idx62)

708259    01901321371111111
Name: 차대번호, dtype: object

In [394]:
# 17자리 X, 영문 : 57
count = 0
p63 = re.compile('[a-zA-Z]+')
idx63 = []
for i, one in tqdm(enumerate(df2.iloc[:, 6].to_numpy())):
    if len(str(one)) != 17 and p63.search(one):
        idx63.append(i)
        count += 1
count

1528586it [00:00, 1764494.52it/s]


57

In [395]:
df2.iloc[:, 6].reindex(idx63)[:5]

34589         SL68T032460
69588         RPS13330414
71114        CN-62-T-2213
91950         KPY30051283
96602    KNADA2322HS00898
Name: 차대번호, dtype: object

In [396]:
# 17자리 X, 영문X : 2573
count = 0
p64 = re.compile('[a-zA-Z]+')
idx64 = []
for i, one in tqdm(enumerate(df2.iloc[:, 6].to_numpy())):
    n = 0
    for j in one:
        if p64.search(j):
            n += 1
    if len(str(one)) != 17 and n == 0:
        idx64.append(i)
        count += 1
count

1528586it [00:11, 129636.60it/s]


2573

In [397]:
df2.iloc[:, 6].reindex(idx64)[:5]

429       119811326107
692     27931130152305
1320      119111327184
1475      119811229792
1544      219211301625
Name: 차대번호, dtype: object

In [398]:
# 차대번호 길이 종류 확인
len_vin = []
for i, one in tqdm(enumerate(df2.iloc[:, 6].to_numpy())):
    len_vin.append(len(str(one)))
set(len_vin)

1528586it [00:01, 1494757.53it/s]


{10, 11, 12, 13, 14, 15, 16, 17}

In [399]:
df2['VIN_length'] = [ len(one) for one in df2['차대번호'] ]

In [400]:
df2['VIN_length'].unique()

array([17, 12, 14, 13, 11, 15, 16, 10], dtype=int64)

In [401]:
IOQ = []
for one in df2['차대번호']:
    if ('I' in one) or ('O' in one) or ('Q' in one):
        IOQ.append('Y')
    else:
        IOQ.append('N')
df2['IOQ'] = IOQ

In [402]:
df2['IOQ'].unique()

array(['N', 'Y'], dtype=object)

In [403]:
# 알파벳 포함
included_alpha = []
count = 0
p65 = re.compile('[a-zA-Z]+')
for i, one in tqdm(enumerate(df2.iloc[:, 6].to_numpy())):
    n = 0
    for j in one:
        if p65.search(j):
            n += 1
    if n == 0:
        included_alpha.append('N')
        count += 1
    else:
        included_alpha.append('Y')
count

1528586it [00:11, 127680.42it/s]


2574

In [404]:
df2['alpha'] = included_alpha

In [405]:
df2.groupby(['VIN_length', 'alpha', 'IOQ'])['차대번호'].count()

VIN_length  alpha  IOQ
10          N      N            1
            Y      N            3
11          N      N          209
            Y      N            7
12          N      N         2132
            Y      N            2
13          N      N          154
            Y      N            3
14          N      N           22
15          N      N           38
            Y      N            2
                   Y            1
16          N      N           17
            Y      N           39
17          N      N            1
            Y      N      1524937
                   Y         1018
Name: 차대번호, dtype: int64

In [406]:
df2.groupby(['VIN_length', 'IOQ'])['차대번호'].count()

VIN_length  IOQ
10          N            4
11          N          216
12          N         2134
13          N          157
14          N           22
15          N           40
            Y            1
16          N           56
17          N      1524938
            Y         1018
Name: 차대번호, dtype: int64

In [407]:
V_I_dict = df2.groupby(['VIN_length', 'IOQ'])['차대번호'].groups

In [408]:
V_I_dict.keys()

dict_keys([(10, 'N'), (11, 'N'), (12, 'N'), (13, 'N'), (14, 'N'), (15, 'N'), (15, 'Y'), (16, 'N'), (17, 'N'), (17, 'Y')])

In [409]:
g = df2.groupby(['VIN_length', 'IOQ'], as_index=False)['차대번호'].count()
g

,VIN_length,IOQ,차대번호
0,10,N,4
1,11,N,216
2,12,N,2134
3,13,N,157
4,14,N,22
5,15,N,40
6,15,Y,1
7,16,N,56
8,17,N,1524938
9,17,Y,1018


In [410]:
V_I_df = g.sort_values(['VIN_length', 'IOQ'], ascending=[True, True])
V_I_df

,VIN_length,IOQ,차대번호
0,10,N,4
1,11,N,216
2,12,N,2134
3,13,N,157
4,14,N,22
5,15,N,40
6,15,Y,1
7,16,N,56
8,17,N,1524938
9,17,Y,1018


In [411]:
for v in range(10, 18):
    for i in ['N', 'Y']:
        try:
            idx = V_I_dict[(v, i)]
            print(f'[VIN_length : {v}, "IOQ"포함 : {i}]')
            print('예시)')
            print(df2.iloc[idx[:3], 6])
            print()
        except:
            pass

[VIN_length : 10, "IOQ"포함 : N]
예시)
274224    FD3S303919
303394    EU13315046
472456    1102253020
Name: 차대번호, dtype: object

[VIN_length : 11, "IOQ"포함 : N]
예시)
10469    01891322131
12281    01901321765
12965    01911322485
Name: 차대번호, dtype: object

[VIN_length : 12, "IOQ"포함 : N]
예시)
429     119811326107
1320    119111327184
1475    119811229792
Name: 차대번호, dtype: object

[VIN_length : 13, "IOQ"포함 : N]
예시)
7610     119213303068다
28217    119213302895다
48405    119213302724다
Name: 차대번호, dtype: object

[VIN_length : 14, "IOQ"포함 : N]
예시)
692       27931130152305
206507    21891130315401
276689    119111200061대서
Name: 차대번호, dtype: object

[VIN_length : 15, "IOQ"포함 : N]
예시)
19514    28-88-112-06218
84146    11-97-113-29992
84224    41-77-111-00110
Name: 차대번호, dtype: object

[VIN_length : 15, "IOQ"포함 : Y]
예시)
103131    12851300018EORN
Name: 차대번호, dtype: object

[VIN_length : 16, "IOQ"포함 : N]
예시)
23971    11-94-112-09987다
96602    KNADA2322HS00898
99552    KNAGA222JS117210
Name: 차대번호, dtype: 

In [417]:
df2.groupby(['VIN_length', '연료'])['차대번호'].count().unstack().T

VIN_length,10,11,12,13,14,15,16,17
연료,,,,,,,,
CNG(압축천연가스),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
LPG(액화석유가스),NaN,NaN,15.0,NaN,NaN,NaN,2.0,26402.0
경유,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1167600.0
기타연료,NaN,NaN,NaN,NaN,NaN,1.0,NaN,37.0
알코올,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
휘발유,4.0,216.0,2119.0,157.0,22.0,40.0,44.0,331915.0


In [418]:
df2.groupby(['VIN_length', '연료'])['차대번호'].count().unstack().T.to_excel('analysis/[G4][차대번호]길이_통계.xlsx')

In [427]:
df2.groupby(['IOQ', '연료'])['차대번호'].count().unstack().T

IOQ,N,Y
연료,,
CNG(압축천연가스),1.0,NaN
LPG(액화석유가스),26415.0,4.0
경유,1167410.0,200.0
기타연료,36.0,2.0
알코올,1.0,NaN
휘발유,333704.0,813.0


In [428]:
df2.groupby(['IOQ', '연료'])['차대번호'].count().unstack().T.to_excel('analysis/[G4][차대번호]IOQ포함여부_통계.xlsx')

#### 차대번호 10번째 자리 문자로 연식 확인

In [429]:
df2['차량연식'].unique()

array([2012, 2007, 2010, 2009, 2000, 2008, 2011, 1999, 1991, 1989, 1997,
       1996, 1993, 1994, 1995, 1988, 2002, 2006, 1992, 1990, 1998, 1987,
       2001, 2016, 2017, 2005, 2018, 1986, 2015, 2013, 2004, 2003, 1968,
       1979, 2014, 2019, 1983, 2020, 1978, 1985, 1981, 1980, 1901, 1984,
       1973], dtype=int64)

In [431]:
df2.groupby(['차량연식', '연료'])['차대번호'].count().unstack()

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
차량연식,,,,,,
1901,NaN,NaN,NaN,NaN,NaN,1.0
1968,NaN,NaN,NaN,NaN,NaN,1.0
1973,NaN,NaN,NaN,NaN,NaN,1.0
1978,NaN,NaN,NaN,NaN,NaN,1.0
1979,NaN,NaN,NaN,NaN,NaN,3.0
1980,NaN,NaN,NaN,NaN,NaN,1.0
1981,NaN,NaN,NaN,NaN,NaN,1.0
1983,NaN,NaN,NaN,NaN,NaN,3.0
1984,NaN,NaN,NaN,NaN,NaN,3.0


In [441]:
df2.groupby(['차량연식', '연료'])['차대번호'].count().unstack().to_excel('analysis/[G4][차량연식]통계.xlsx')

In [432]:
df2['차대번호_10자리'] = df2['차대번호'].str[9]
df2['차대번호_10자리'].unique()

array(['C', '7', 'A', '9', 'Y', '8', 'B', 'X', 'M', 'K', 'V', 'T', 'P',
       'R', 'S', 'J', '2', '6', 'N', 'L', 'W', 'H', '1', 'G', '5', '0',
       '4', 'F', '3', 'D', '-', 'E', 'U', 'O'], dtype=object)

In [433]:
vin_10_dict = {'A':1980, 'B':1981, 'C':1982, 'D':1983, 'E':1984, 'F':1985, 'G':1986, 'H':1987, 'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009}

In [434]:
df2['차대번호_10자리_기준연식'] = df2['차대번호_10자리'].replace(vin_10_dict)

In [435]:
df2['차대번호_10자리_기준연식'].unique()

array([1982, 2007, 1980, 2009, 2000, 2008, 1981, 1999, 1991, 1989, 1997,
       1996, 1993, 1994, 1995, 1988, 2002, 2006, 1992, 1990, 1998, 1987,
       2001, 1986, 2005, '0', 2004, 1985, 2003, 1983, '-', 1984, 'U', 'O'],
      dtype=object)

In [437]:
df2.groupby(['차대번호_10자리_기준연식', '연료'])['차대번호'].count().unstack()

연료,CNG(압축천연가스),LPG(액화석유가스),경유,기타연료,알코올,휘발유
차대번호_10자리_기준연식,,,,,,
1980,NaN,NaN,252706.0,NaN,NaN,2.0
1981,NaN,NaN,237573.0,NaN,NaN,5.0
1982,NaN,NaN,94135.0,NaN,NaN,54.0
1983,NaN,NaN,29.0,NaN,NaN,70.0
1984,NaN,NaN,13.0,NaN,NaN,1.0
1985,NaN,NaN,463.0,NaN,NaN,6.0
1986,NaN,NaN,482.0,NaN,NaN,12.0
1987,NaN,NaN,732.0,NaN,NaN,1299.0
1988,NaN,91.0,484.0,9.0,1.0,11646.0


In [440]:
df2.groupby(['차대번호_10자리_기준연식', '연료'])['차대번호'].count().unstack().to_excel('analysis/[G4][차량연식]차대번호10번째자리문자기준_통계.xlsx')

# 코드 마지막